In [2]:
import pandas as pd


In [3]:

songs = pd.read_csv('tracks.csv')
streams = pd.read_csv('streams.csv')
# Prepare data
streams['listen_date'] = pd.to_datetime(streams['listen_time']).dt.date
merged_data = streams.merge(songs, on='track_id', how='left')

# Compute each KPI
# KPI 1: Daily Genre Listen Count
genre_listen_count = merged_data.groupby(['listen_date', 'track_genre']).size().reset_index(name='listen_count')

# KPI 2: Average Listening Duration per Genre per Day
merged_data['duration_seconds'] = merged_data['duration_ms'] / 1000
avg_duration = merged_data.groupby(['listen_date', 'track_genre'])['duration_seconds'].mean().reset_index(name='average_duration')

# KPI 3: Daily Genre Popularity Index
total_listens = merged_data.groupby('listen_date').size().reset_index(name='total_listens')
genre_listen_count = genre_listen_count.merge(total_listens, on='listen_date')
genre_listen_count['popularity_index'] = genre_listen_count['listen_count'] / genre_listen_count['total_listens']

# KPI 4: Most Popular Track per Genre per Day
most_popular_track = merged_data.groupby(['listen_date', 'track_genre', 'track_id']).size().reset_index(name='track_count')
most_popular_track = most_popular_track.sort_values(by=['listen_date', 'track_genre', 'track_count'], ascending=[True, True, False])
most_popular_track = most_popular_track.drop_duplicates(subset=['listen_date', 'track_genre'], keep='first').rename(columns={'track_id': 'most_popular_track_id'})

# Combine all KPIs into one DataFrame
final_kpis = genre_listen_count[['listen_date', 'track_genre', 'listen_count', 'popularity_index']]
final_kpis = final_kpis.merge(avg_duration, on=['listen_date', 'track_genre'])
final_kpis = final_kpis.merge(most_popular_track[['listen_date', 'track_genre', 'most_popular_track_id']], on=['listen_date', 'track_genre'])

 
 
 

In [4]:
final_kpis.head()


,listen_date,track_genre,listen_count,popularity_index,average_duration,most_popular_track_id
0,2024-01-01,Classical,210,0.198675,214.455038,5ety2MTWwnJxPjjeQLU7f0
1,2024-01-01,Electronic,171,0.161779,204.176175,0hL56sGZwCECqR1CMZ3sFy
2,2024-01-01,Hip Hop,176,0.166509,214.409915,2HsRx4mjgCii6DBWCJch86
3,2024-01-01,Jazz,191,0.180700,215.463466,YBtRN6gZpUCPyTOBoJnaN1
4,2024-01-01,Pop,173,0.163671,211.683699,2DyPgioORYaJIJ2PBMzbUa
